
# Modelos y Simulación – **Clase 1**
## Introducción a la Inferencia Estadística 

**Autor:** Cátedra – Modelos y Simulación - Facultad de Ingeniería - UCA - Javier Ouret   
**Fecha:** 2025-08-09

---



#### Módulo de utilidades estadísticas para medias. 
Funciones para pruebas de hipótesis sobre medias (Z y t), intervalos de confianza y potencia aproximada.  

##### Cuantil de la normal: inv_norm_cdf(p) 

Devuelve z tal que Φ(z)=p (inversa de la CDF normal estándar). Se usa para obtener críticos Z.   
$\Phi(z)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{z} e^{-t^2/2}\,dt,
\qquad z = \Phi^{-1}(p)$.   

##### Valor Crítico Z bilateral: z_crit(alpha_two_sided=0.05)   

Devuelve z1−α/2. Para α=0.05, z0.975≈1.96z  
$z_{\text{crit}} = z_{1-\alpha/2}.$

##### Valor Crítico t aproximado: tcrit_approx_two_sided(df, alpha=0.05)

- Si α=0.05 y los grados de libertad ν=df están en la tabla incluida, devuelve t1−α/2, ν
- Si no está, toma el df más cercano.
- Si α≠0.05, hace fallback a z1−α/2 (aproximación).   
$t_{\text{crit}} \approx t_{1-\alpha/2,\ \nu}
\quad\text{(o bien)}\quad
t_{\text{crit}} \approx z_{1-\alpha/2}\ \text{si no hay tabla para }\alpha$.

##### IC para la media con σ desconocida: ci_mean_unknown_sigma(xbar, s, n, alpha)

Devuelve (LI,LS,tcrit) del intervalo t.

$\text{IC}_{1-\alpha}(\mu):\ \ 
\bar{X} \pm t_{1-\alpha/2,\ n-1}\,\frac{s}{\sqrt{n}},
\qquad
\text{semi-ancho} = t_{1-\alpha/2,\ n-1}\,\frac{s}{\sqrt{n}}$.
##### Estadístico de prueba t: t_statistic(xbar, mu0, s, n)

Calcula el t de Student para contrastar H0:μ=μ0 con σ desconocida.

$t = \frac{\bar{X} - \mu_0}{s/\sqrt{n}}, 
\qquad t \sim t_{n-1}\ \ \text{bajo }H_0$.
##### Potencia aproximada (bilateral, normal): approx_power_two_sided(mu0, mu1, s, n, alpha)

Aproxima la potencia usando normal (no t no-central). Define el tamaño de efecto estandarizado

$\Delta = \frac{\mu_1 - \mu_0}{s/\sqrt{n}}, 
\qquad z_c = z_{1-\alpha/2}$.   
y usa   
$\text{Potencia} \approx 
\big[1-\Phi(z_c - \Delta)\big] + \Phi(-z_c - \Delta)$.

##### Se llama valor crítico porque es el valor del estadístico de prueba (Z, t, χ², F, etc.) que marca la frontera entre la región de aceptación y la región de rechazo de la hipótesis nula.
- Si el estadístico calculado es mayor (en valor absoluto) que el valor crítico, caemos en la región de rechazo → se rechaza H0.
- Si es menor, no se rechaza H0.

In [12]:

import math, statistics, random
from statistics import mean, stdev

def inv_norm_cdf(p):
    if p <= 0.0 or p >= 1.0:
        raise ValueError("p must be in (0,1)")
    a = [-3.969683028665376e+01,  2.209460984245205e+02,
         -2.759285104469687e+02,  1.383577518672690e+02,
         -3.066479806614716e+01,  2.506628277459239e+00]
    b = [-5.447609879822406e+01,  1.615858368580409e+02,
         -1.556989798598866e+02,  6.680131188771972e+01,
         -1.328068155288572e+01]
    c = [-7.784894002430293e-03, -3.223964580411365e-01,
         -2.400758277161838e+00, -2.549732539343734e+00,
          4.374664141464968e+00,  2.938163982698783e+00]
    d = [ 7.784695709041462e-03,  3.224671290700398e-01,
          2.445134137142996e+00,  3.754408661907416e+00]
    plow = 0.02425
    phigh = 1 - plow
    if p < plow:
        q = math.sqrt(-2*math.log(p))
        return (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
               ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)
    if phigh < p:
        q = math.sqrt(-2*math.log(1-p))
        return -(((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
                 ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)
    q = p - 0.5
    r = q*q
    a0 = (((((a[0]*r + a[1])*r + a[2])*r + a[3])*r + a[4])*r + a[5])
    b0 = (((((b[0]*r + b[1])*r + b[2])*r + b[3])*r + b[4])*r + 1)
    return (a0*q) / b0

def z_crit(alpha_two_sided=0.05):
    return abs(inv_norm_cdf(1 - alpha_two_sided/2))

t_crit_table_95 = {
    1:12.706, 2:4.303, 3:3.182, 4:2.776, 5:2.571, 6:2.447, 7:2.365, 8:2.306,
    9:2.262, 10:2.228, 11:2.201, 12:2.179, 13:2.160, 14:2.145, 15:2.131,
    16:2.120, 17:2.110, 18:2.101, 19:2.093, 20:2.086, 21:2.080, 22:2.074,
    23:2.069, 24:2.064, 25:2.060, 26:2.056, 27:2.052, 28:2.048, 29:2.045,
    30:2.042, 40:2.021, 60:2.000, 120:1.980
}

def tcrit_approx_two_sided(df, alpha=0.05):
    if df in t_crit_table_95 and abs(alpha-0.05) < 1e-9:
        return t_crit_table_95[df]
    nearest = min(t_crit_table_95.keys(), key=lambda k: abs(k-df))
    if abs(alpha-0.05) < 1e-9:
        return t_crit_table_95[nearest]
    return z_crit(alpha)

def ci_mean_unknown_sigma(xbar, s, n, alpha=0.05):
    df = n-1
    tcrit = tcrit_approx_two_sided(df, alpha)
    half = tcrit * (s / math.sqrt(n))
    return (xbar - half, xbar + half, tcrit)

def t_statistic(xbar, mu0, s, n):
    return (xbar - mu0) / (s / math.sqrt(n))

def approx_power_two_sided(mu0, mu1, s, n, alpha=0.05):
    zc = z_crit(alpha)
    Delta = (mu1 - mu0) / (s / math.sqrt(n))
    from math import erf, sqrt
    def Phi(z):
        return 0.5*(1 + erf(z/math.sqrt(2)))
    power = (1 - Phi(zc - Delta)) + Phi(-zc - Delta)
    return power



### Caso de uso: **espesores** de perfiles galvanizados.
- **IC**, **prueba t**, **curva de potencia** y **cálculo de tamaño muestral**.
- Funciones reutilizables para:
  - obtener críticos Z/t,
  - calcular IC de la media con σσ desconocida,
  - computar el estadístico t,
  - estimar la potencia de una prueba bilateral.



Parámetros base

- Espesor nominal
- Tamaño muestral
- Nivel


In [5]:

import random, statistics

mu0 = 0.90      # mm
n   = 25        # tamaño muestral
alpha = 0.05    # 5%

# Datos simulados (editables). Reemplazalos por tus mediciones reales si querés.
random.seed(123)
data = [0.92 + random.gauss(0, 0.03) for _ in range(n)]

xbar = sum(data)/n
s = statistics.stdev(data)
xbar, s, n, mu0, alpha


(0.9250663891742161, 0.020618592933675864, 25, 0.9, 0.05)


## Agregar datos

Lista de espesores en mm separados por comas, por ejemplo:
```
0.91, 0.92, 0.89, 0.94, 0.90, 0.93
```
Ejecutar la celda para reemplazar `data`, recalcular $(\bar{x}), (s)$, IC y test.


In [10]:

# Pega/edita aquí (entre las comillas) tu lista de lecturas separadas por coma:
raw = ""
if raw.strip():
    try:
        data = [float(tok) for tok in raw.split(",") if tok.strip()!=""]
        n = len(data)
        import statistics
        xbar = sum(data)/n
        s = statistics.stdev(data) if n>1 else float('nan')
        print(f"Datos cargados: n={n}, x̄={xbar:.4f} mm, s={s:.4f} mm")
    except Exception as e:
        print("Error al parsear tus datos:", e)
else:
    print("No se pegaron datos; se mantienen los simulados.")


No se pegaron datos; se mantienen los simulados.



## Intervalo de confianza y prueba t

Calculamos IC (con $(\sigma)$ desconocida) y test t bilateral $(H_0:\mu=\mu_0)$.


In [7]:

ci_low, ci_high, tcrit = ci_mean_unknown_sigma(xbar, s, n, alpha)
t_stat = t_statistic(xbar, mu0, s, n)
decision = "Rechazar H0" if abs(t_stat) > tcrit else "No rechazar H0"

print(f"Media muestral (x̄): {xbar:.4f} mm")
print(f"Desvío muestral (s): {s:.4f} mm")
print(f"IC {(1-alpha)*100:.1f}%: ({ci_low:.4f}, {ci_high:.4f})  [t_crit≈{tcrit:.3f}]")
print(f"t = {t_stat:.3f}  | Regla: rechazar si |t| > {tcrit:.3f}")
print(f"Decisión: {decision}")


Media muestral (x̄): 0.9251 mm
Desvío muestral (s): 0.0206 mm
IC 95.0%: (0.9166, 0.9336)  [t_crit≈2.064]
t = 6.079  | Regla: rechazar si |t| > 2.064
Decisión: Rechazar H0



## Curva de potencia (aproximación)

Potencia del test bilateral para detectar un desvío mínimo relevante $(\delta)$.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

delta = 0.02  # mm
n_values = list(range(8, 81, 4))

powers = [approx_power_two_sided(mu0, mu0+delta, s, n_i, alpha) for n_i in n_values]

plt.figure()
plt.plot(n_values, powers, marker='o')
plt.xlabel("Tamaño muestral n")
plt.ylabel("Potencia aproximada")
plt.title(f"Curva de potencia (α={alpha}, δ={delta:.3f} mm)")
plt.grid(True)
plt.show()



## ¿Qué n necesito para una potencia objetivo?

Sea $(\delta = |\mu_1-\mu_0|)$. Hallamos el $(n)$ mínimo (aprox.) para potencia $(\ge 1-\beta)$.


In [11]:

target_power = 0.8  # 80%
delta = 0.02        # mm

n_min = None
for n_i in range(8, 1000):
    p = approx_power_two_sided(mu0, mu0+delta, s, n_i, alpha)
    if p >= target_power:
        n_min = n_i
        break

print(f"Objetivo: potencia ≥ {target_power:.2f} con δ={delta:.3f} mm, α={alpha}")
print("Resultado:", "no alcanzado" if n_min is None else f"n mínimo ≈ {n_min}")


Objetivo: potencia ≥ 0.80 con δ=0.020 mm, α=0.05
Resultado: n mínimo ≈ 9
